In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import matplotlib

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.1
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.97
plt.rcParams['figure.subplot.bottom'] = 0.05
plt.rcParams['figure.subplot.hspace'] = 0.3
matplotlib.rc("image", cmap="gray_r")

In [2]:
%matplotlib qt

In [3]:
real_image= cv2.imread(
r"D:\WORKS\TechTasks\DocData\2.jpg"
)
plt.imshow(real_image)

# Prepare

In [4]:
gray_image = cv2.cvtColor(real_image, cv2.COLOR_BGR2GRAY)
gray_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
plt.imshow(gray_image)

In [5]:
def downscale_image(image, max_size=2048):
    """Shrink im until its longest dimension is <= max_dim.

    Returns new_image, scale (where scale <= 1).
    """

    x, y = image.shape
    max_dimention = max(x, y)
    if max_dimention <= max_size:
        return image

    scale = max_size / max_dimention
    return cv2.resize(
        image, None, fx= scale, fy= scale, interpolation=cv2.INTER_LINEAR
    )


gray_image = downscale_image(gray_image)
plt.imshow(gray_image)

# Find text boxes

In [6]:
from scipy.ndimage import rank_filter
from typing import NamedTuple

In [38]:
class BlockTuple(NamedTuple):
    i: int
    power: int
    mask: np.ndarray


a = gray_image
a = cv2.GaussianBlur(a, (5, 5), 5)
a = cv2.Canny(a, 100, 200)
a = rank_filter(a, rank=5, size=3)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
n_rectangles = 1000
while n_rectangles > 20:
    a = cv2.dilate(a, kernel, iterations=1)
    n_rectangles, block_map = cv2.connectedComponents(a)
    n_rectangles -= 1

In [39]:
ranged_blocks: list[BlockTuple] = sorted(
    [
        BlockTuple(
            i,
            np.count_nonzero((block_map == i) * 1),
            np.uint8(block_map == i),
        )
        for i in range(1, n_rectangles)
    ],
    key=lambda x: x.power,
)
print(n_rectangles - 1)
# a = cv2.dilate(a, kernel, iterations=5)
plt.imshow(block_map)

15


In [42]:
def watch(mask, image):
    border_scale = 0.05
    y_full, x_full = image.shape
    y_border = int(y_full * border_scale)
    x_border = int(x_full * border_scale)
    non_zero_coords = cv2.findNonZero(mask)
    x, y, dx, dy = cv2.boundingRect(non_zero_coords)
    window_slice = (
        slice(y - y_border, y + y_border + dy),
        slice(x - x_border, x + x_border + dx),
    )
    window = np.zeros_like(image)
    window[window_slice] = image[window_slice]
    return window, (dx, dy)


def F_metric(mask, window, size):
    mask = mask.copy()
    mask[mask != 0] = 1
    window = window.copy()
    window[window != 0] = 1

    dx, dy = size

    tp = np.count_nonzero(mask)
    fp = dx * dy
    fn = np.count_nonzero(window - mask)
    return 2 * tp / (2 * tp + fp + fn)

In [45]:
paragraphs = []
while len(ranged_blocks) > 13:
    focused = ranged_blocks.pop()
    focused = focused.mask.copy()
    m0, img, bord = F_metric(focused, block_map)

    watched_blocks = [block for block in ranged_blocks if block.i in set(np.unique(bord))]
    # window = current + sum([block.mask for block in watched_blocks])

    while len(watched_blocks) > 0:
        add_block = watched_blocks.pop()
        m, img, bord = F_metric(focused + add_block.mask, block_map)
        if m >= m0*0.9 :
            m0 = m
            focused += add_block.mask
            ranged_blocks.remove(add_block)

    paragraphs.append(focused)

print(m)
plt.imshow(focused)

0.41581083352677967


In [33]:
focused = ranged_blocks.pop().mask.copy()
window, size = watch(focused, block_map)
m0 = F_metric(focused, window,size)

watched_blocks = [block for block in ranged_blocks if block.i in set(np.unique(window))]
print(len(watched_blocks))
plt.imshow(focused * 10 + window)

IndexError: pop from empty list

In [30]:
add_block = watched_blocks.pop()
focused_new = focused + add_block.mask
window, size = watch(focused_new, block_map)
m = F_metric(focused_new, window, size)
if m >= m0 *0.9:
    m0 = m
    focused = focused_new
    ranged_blocks.remove(add_block)
print(m)
plt.imshow(focused * 10 + window)

IndexError: pop from empty list

In [43]:
def select_near(focused, watched_blocks,ranged_blocks):
    m0 = F_metric(focused, window, size)
    while len(watched_blocks):

        add_block = watched_blocks.pop()
        focused_new = focused + add_block.mask
        window, size = watch(focused_new, block_map)
        m = F_metric(focused_new, window, size)
        if m >= m0 * 0.9:
            m0 = m
            focused = focused_new
            ranged_blocks.remove(add_block)
    return focused_new

In [44]:
paragpaph_list = []
while len(ranged_blocks):
    focused = ranged_blocks.pop().mask.copy()
    window, size = watch(focused, block_map)


    watched_blocks = [block for block in ranged_blocks if block.i in set(np.unique(window))]
    paragpaph_list.append(select_near(focused, watched_blocks, ranged_blocks))

UnboundLocalError: local variable 'm0' referenced before assignment

In [204]:
plt.imshow(focused_new)

In [57]:
while len(watched_blocks) > 0:
    add_block = watched_blocks.pop()
    m, img, bord = F_metric(current + add_block.mask, block_map)
    if m >= m0*0.9 :
        m0 = m
        current += add_block.mask
        ranged_blocks.remove(add_block)
    input()
    plt.imshow(img*10 + bord)


In [36]:
paragpaph_list

[]

In [35]:
plt.imshow(sum([paragpaph_list[i] * (i+1) for i in range(len(paragpaph_list))]))

TypeError: Invalid shape () for image data

In [35]:
plt.imshow(block_map)

In [ ]:
def get_min_area_rect(image):
    non_zero_coords = cv2.findNonZero(image)
    return cv2.minAreaRect(non_zero_coords)
